# DINEOF Preprocessing of data

In this script we will:

* Read initial data file
* Create land-sea mask
* Eliminate pixels/images that are too often cloudy
* Create time variable
* Write results to a new file to be used in DINEOF

In [ ]:
#import Pkg; Pkg.add("Missings")
using NCDatasets
using PyPlot
using Missings
using Dates
using Statistics
include("dineof_scripts.jl")

In [ ]:
#We will download the data
filename = "METEOFRANCE-EUR-SST_L3MULTISENSOR_NRT-OBS_FULL_TIME_SERIE_1634307629210_adriatic_long.nc";
if !isfile(filename)
    @info("downloading $filename")
    cp(download("https://dox.uliege.be/index.php/s/nzfdz6ESOQIEthz/download"),filename)
else
    @info("$filename is already downloaded")
end


In [ ]:
#Reading Sentinel-3A  SST L3 data as downloaded from CMEMS site
#ds = Dataset("METEOFRANCE-EUR-SST_L3MULTISENSOR_NRT-OBS_FULL_TIME_SERIE_1634307629210_adriatic_long.nc");

#A
ds = Dataset(filename);

tmp = nomissing(ds["sea_surface_temperature"][:],NaN);
sst  = tmp .- 273.15;
time = ds["time"][:]; 
lat = ds["lat"][:];
lon = ds["lon"][:];
tmp2 = ds["source_of_sst"][:];

tmp2 = Array{Union{Missing,Float64}}(tmp2);
tmp2[ismissing.(tmp2)].=NaN;

# We remove certain satellite sensors which are coarser
sst[tmp2.==14].=NaN; #AMSRE
sst[tmp2.==15].=NaN; #TMI
sst[tmp2.==29].=NaN; #SEVIRI
#Size of SST dataset
@show size(tmp)
close(ds);

In [ ]:
# We are retaining only ~3 months of data in order to make the run faster (it's just a test...). 
# But if you want to work with the full 1 year of data, comment the following lines

sst = sst[:,:,150:365];
tmp = tmp[:,:,150:365];
time = time[150:365];

In [ ]:
#Start and end dates of our dataset

@show time[1]
@show time[end];
@show typeof(tmp2)
@show typeof(sst);

In [ ]:
#Quick visualisation of one image with and without the removed satellite sensors
i=4;
figure()
pcolor(lon',lat,tmp[:,:,i]'.-273.15,cmap="RdYlBu_r");clim(8,17);colorbar()#(orientation="horizontal");
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);

title("SST all sensors")
@show(size(sst))

figure()
pcolor(lon',lat,sst[:,:,i]',cmap="RdYlBu_r");clim(8,17);colorbar()#(orientation="horizontal")
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);

title("SST $(time[i])")
@show size(sst);


In [ ]:
#The domain contains regions (Tyrrhenian Sea, Ionian Sea) that are not of interest for us
pcolor(sst[:,:,92]',cmap="RdYlBu_r");clim(18,28);colorbar();

In [ ]:
# We will remove any sea part that is not our zone of interest
# with a "low-tech" approach; just choose those regions by their indices (trial and error)
sst[1:205,1:125,:].=NaN;
sst[1:50,100:160,:].=NaN;
sst[200:275,1:78,:].=NaN;
sst[275:300,1:70,:].=NaN;
sst[300:401,1:60,:].=NaN;
pcolor(sst[:,:,92]',cmap="RdYlBu_r");clim(18,28);colorbar();

In [ ]:
#transform time variable (in miliseconds) into year-day 
mdate = Dates.value.(time - DateTime(2017,1,1))/1000/60/60/24;

#create a first land-sea mask. This mask will be "refined" by eliminating pixels that are 
# covered more than 98% of the time and images that are covered more than 98% in space
mask = nanmean(sst,3);
mask[.!isnan.(mask)].=1;
mask[isnan.(mask)].=0;

covT = coverage(sst,mask,"tm"); #calculate average % of missing data in time

#Visualise % of missing data in time
plot(covT)
println("Average amount of missing data in your dataset: $(mean(covT)) %");

In [ ]:
#There are images with almost no data. We will eliminate those

i=findall(covT.<98); #identify images with more than 95% of missing data
sstb = sst[:,:,i]; #remove those images
mdateb = mdate[i]; #remove those dates from the time vector

#old and new temporal size of the SST matrix
@show(size(sst))
@show size(sstb);

In [ ]:
#We will now do a land-sea mask by finding values with no data (=land)

maskb = nanmean(sstb,3); #new mask with the new matrix
maskb[.!isnan.(maskb)].=1; #non-missing data to sea
maskb[isnan.(maskb)].=0; #missing data to land
@show size(maskb)
covS = coverage(sstb,maskb,"sp");  #calculate average % of missing data in space

#Visualise the % of missing data in space
pcolor(lon',lat,covS'),clim(50,100),colorbar()#(orientation="horizontal");
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);


The regions most affected by missing data are the coastal ones, especially along the Croatian coast due to the large amount of islands.

In [ ]:
#maximum value of missing data 

extrema(covS)

In [ ]:
#As we did with the spatial average amount of missing data, we will now remove points that are missing >98% of the time

covS[covS.>=98].=0; #if a pixel is missing more than 98% of the time we set that to land
@show extrema(covS)

#the map showing % of missing data will be transformed in the final land-sea mask
covS[covS.==0].=NaN;

covS[.!isnan.(covS)].=1; #non-missing data to sea

covS[isnan.(covS)].=0; #missing data to land


#Land-sea mask
pcolor(lon',lat,covS',cmap="RdYlBu_r");colorbar()
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);


In [ ]:
#We visualise a last example
contourf(lon',lat,covS',levels = [0., 0.5],colors =[[.5,.5,.5]])
pcolor(lon',lat,sstb[:,:,92]',cmap="RdYlBu_r");clim(18,28),colorbar()#(orientation="horizontal")
ylim(40,46)
aspect_ratio=1/cos(pi*mean(lat)/180);
gca().set_aspect(aspect_ratio);


In [ ]:
#write down the results into a new netCDF file
output = Dataset("sst_L3_Adriatic.nc","c");
defDim(output,"lon",size(maskb,1))
defDim(output,"lat",size(maskb,2))
defDim(output,"time",size(sstb,3))

ncCHL = defVar(output,"SST",Float32,("lon","lat","time");fillvalue=-9999.f0);
sstb[isnan.(sstb)].=-9999.;
ncCHL[:] = sstb;

ncTime = defVar(output,"time",Float32,("time",));
ncTime[:] = mdateb;

ncMask = defVar(output,"mask",Float32,("lon","lat"));
ncMask[:] = covS;

ncLat = defVar(output,"lat",Float32,("lat",));
ncLat[:] = lat;

ncLon = defVar(output,"lon",Float32,("lon",));
ncLon[:] = lon;

close(output)

In [ ]:
#Choose cross-validation points in the form of real clouds
#You should run changing last argument until % of added clouds is about 3%
#output is in clouds_index.nc, but should be renamed to avoid overwriting it.
include("dineof_scripts.jl")
dineof_cvp("sst_L3_Adriatic.nc#SST","sst_L3_Adriatic.nc#mask",".",3);